## 설정 초기화

In [18]:
%pip install mysql-connector-python

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [40]:
import mysql.connector
from mysql.connector import Error


connection = None

try:
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        database='cnu_data',
        user='root',
        password='dydrkfl#7!'
    )
    
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"MySQL 서버에 연결되었습니다. 버전: {db_info}")
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print(f"현재 데이터베이스: {record[0]}")

except Error as e:
    print(f"MySQL 연결 중 에러 발생: {e}")


MySQL 서버에 연결되었습니다. 버전: 9.3.0
현재 데이터베이스: cnu_data


/var/folders/kv/jv4ntr9x5h3f_0rbw1xytqbm0000gp/T/ipykernel_81382/3892527287.py:17: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()


In [4]:
import requests
import json
import datetime 
def log_api_error(endpoint, error_message):
    """
    API 호출 에러를 로그 파일에 기록하는 함수
    
    Args:
        endpoint (str): API 엔드포인트 URL
        error_message (str): 발생한 에러 메시지
    """
    try:
        with open('api_log.log', 'a', encoding='utf-8') as log_file:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_entry = f"[{timestamp}] API 호출 실패\n"
            log_entry += f"엔드포인트: {endpoint}\n"
            log_entry += f"에러 메시지: {error_message}\n"
            log_entry += "-" * 80 + "\n"
            log_file.write(log_entry)
    except Exception as e:
        print(f"로그 파일 작성 중 에러 발생: {e}")

def fetch_data(endpoint) -> list:
    """
    CNU API에서 데이터를 가져오는 함수
    
    Args:
        endpoint (str): API 엔드포인트 URL
        
    Returns:
        dict: API 응답 데이터
    """
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    response = requests.post(endpoint, headers=headers)
    response.raise_for_status()
    
    # 제어 문자 필터링
    # - ASCII 32 미만의 제어문자 제거 (\n, \r, \t 제외)
    # - ASCII 127 (DEL) 제거
    # - ASCII 128-159 사이의 제어문자 제거
    # - 유니코드 한글 범위 (AC00-D7AF) 허용
    # - 기본 ASCII 문자 (32-126) 허용
    cleaned_text = ''.join(ch for ch in response.text 
                            if (ord(ch) >= 32 and ord(ch) <= 126) # 기본 ASCII
                            or ch in {'\n', '\r', '\t'} # 허용된 제어문자
                            or (0xAC00 <= ord(ch) <= 0xD7AF) # 한글 유니코드
                            or (ord(ch) > 159)) # 그 외 유니코드 문자
    
    data = json.loads(cleaned_text)
    print(f"API 응답 상태: {response.status_code}")
    
    # MSG 필드가 success를 포함하는지 확인
    if ('OutBlock' not in data and 'outBlock' not in data) or \
   (not data.get('OutBlock') and not data.get('outBlock')) or \
   ('MSG' not in (data.get('OutBlock', [{}])[0] if data.get('OutBlock') else data.get('outBlock', [{}])[0])) or \
   'success' not in ((data.get('OutBlock', [{}])[0].get('MSG', '') if data.get('OutBlock') else data.get('outBlock', [{}])[0].get('MSG', ''))):
       raise Exception(f"API 응답 실패 in {endpoint}: {data}")
    
    return data["RESULT"]
    

import csv
def dict_to_csv(data: dict, output_file_name: str):
    """
    딕셔너리 형태의 데이터를 CSV 파일로 변환하는 함수
    
    Args:
        data (dict): 변환할 딕셔너리 데이터. 모든 value는 같은 길이의 리스트여야 함
        output_file (str): 저장할 CSV 파일명
    """
    try:
        # 데이터가 비어있는지 확인
        if not data or not all(isinstance(v, list) for v in data.values()):
            raise ValueError("데이터가 올바른 형식이 아닙니다.")
            
        # 모든 리스트의 길이가 같은지 확인
        list_lengths = [len(v) for v in data.values()]
        if len(set(list_lengths)) > 1:
            raise ValueError("모든 리스트의 길이가 같아야 합니다.")
            
        # CSV 파일 생성
        with open(f"{output_file_name}.csv", 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # 헤더(컬럼명) 작성
            writer.writerow(data.keys())
            
            # 데이터 작성
            rows = zip(*data.values())
            writer.writerows(rows)
            
        print(f"CSV 파일이 성공적으로 생성되었습니다: {output_file_name}")
        
    except Exception as e:
        print(f"CSV 파일 생성 중 에러 발생: {e}")

# TODO: 수강신청 데이터 전처리 전략 세우기(수강신청을 얼마나 많이 했는지 등)

In [3]:
AUTH_KEY = '081E3B5B5C5E47409E3AB0BE3FBFC9FDC7798045'
CNU_API_ENDPOINT = 'https://api.cnu.ac.kr/svc/offcam/pub'

### 강의계획서

In [16]:
# 강의계획서 테이블 생성
try:
    if connection.is_connected():
        cursor = connection.cursor()
        
        create_table_query = """
        CREATE TABLE IF NOT EXISTS lecture_plan (
            id INT AUTO_INCREMENT PRIMARY KEY,
            year INT NOT NULL,
            semester VARCHAR(10) NOT NULL,
            target_year INT,
            organization_type VARCHAR(20),
            college VARCHAR(50),
            department VARCHAR(100),
            subject_no VARCHAR(20),
            subject_name VARCHAR(200),
            class_division VARCHAR(10),
            subject_type VARCHAR(50),
            credit INT,
            theory_hours INT,
            practice_hours INT,
            summary TEXT,
            subject_description TEXT,
            textbook TEXT,
            reference_book TEXT,
            prerequisite TEXT,
            professor VARCHAR(100),
            timetable TEXT,
            lesson_type VARCHAR(50),
            evaluation_method VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        cursor.execute(create_table_query)
        connection.commit()
        print("강의계획서 테이블이 생성되었습니다.")

except Exception as e:
    print(f"테이블 생성 중 오류 발생: {str(e)}")

# 데이터 저장
insert_query = """
INSERT INTO lecture_plan (
    year, semester, target_year, organization_type, college, department,
    subject_no, subject_name, class_division, subject_type, credit,
    theory_hours, practice_hours, summary, subject_description,
    textbook, reference_book, prerequisite, professor, timetable,
    lesson_type, evaluation_method
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s
)
"""

강의계획서 테이블이 생성되었습니다.


In [ ]:
# 데이터 수집 및 저장

# 진행 상태를 저장/불러오기 위한 파일명
PROGRESS_FILE = 'lecture_scraping_progress.txt'

# 이전 진행상태 불러오기
def load_progress():
    try:
        with open(PROGRESS_FILE, 'r') as f:
            year, semester, page, data_idx = map(int, f.read().strip().split(','))
        return year, semester, page, data_idx
    except FileNotFoundError:
        return 2022, 1, 1, 0  # 초기값

# 현재 진행상태 저장
def save_progress(year, semester, page, data_idx):
    with open(PROGRESS_FILE, 'w') as f:
        f.write(f"{year},{semester},{page},{data_idx}")

# 저장된 진행상태 불러오기
current_year, current_semester, current_page, current_data_idx = load_progress()

for year in range(current_year, 2025+1):
    for semester in range(current_semester, 2+1):
        page = current_page if year == current_year and semester == current_semester else 1
        while True:
            endpoint = f'{CNU_API_ENDPOINT}/lsnSmry?AUTH_KEY={AUTH_KEY}&P_YR={year}&P_OPEN_SHTM_CD={semester}&page={page}'
            
            # API 호출
            data = fetch_data(endpoint)
            
            if not data:
                break
                
            try:     
                start_idx = current_data_idx if year == current_year and semester == current_semester and page == current_page else 0
                for idx, lecture in enumerate(data[start_idx:], start=start_idx):
                    values = (
                        lecture['OPEN_YR'],
                        lecture['SHTM'],
                        lecture['TRGT_SHYR'],
                        lecture['ORGN_CLSF_CD'],
                        lecture['COLG'],
                        lecture['DEGR_NM_SUST'],
                        lecture['OPEN_SBJT_NO'],
                        lecture['OPEN_SBJT_NM'],
                        lecture['OPEN_DCLSS'],
                        lecture['CPTN_DIV_NM'],
                        lecture['PNT'],
                        lecture['THEO_TMCNT'],
                        lecture['PRAC_TMCNT'],
                        lecture['LSN_SMRY'],
                        lecture['SBJT_SHT'],
                        lecture['TEMT_REF_LITRT'],
                        lecture['REF_BOOK'],
                        lecture['PRE_LRN_CN'],
                        lecture['PROF_INFO'],
                        lecture['TMTBL_INFO'],
                        lecture['LSN_TYPE_NM'],
                        lecture['MRKS_EVL_MTHD_NM']
                    )
                    cursor.execute(insert_query, values)
                    # 현재 진행상태 저장
                    save_progress(year, semester, page, idx + 1)
                
                connection.commit()
                print(f"{year}년 {semester}학기 {page}페이지 데이터 저장 완료")
                
                # 페이지 정보 확인
                if page >= int(data[0]['PAGECNT']):
                    current_data_idx = 0  # 다음 학기/연도로 넘어갈 때 초기화
                    break
                    
                page += 1
                current_data_idx = 0  # 다음 페이지로 넘어갈 때 초기화
                
            except Exception as e:
                print(f"데이터 저장 중 오류 발생: {str(e)}")
                # 오류 발생 시 현재 진행상태가 저장되어 있으므로 다음 실행 시 이어서 진행 가능
                break
        current_semester = 1  # 다음 연도로 넘어갈 때 학기 초기화
    current_page = 1  # 다음 연도로 넘어갈 때 페이지 초기화

### CNU 게시판 목록

In [ ]:
endpoint = f'{CNU_API_ENDPOINT}/cmsBoard?AUTH_KEY={AUTH_KEY}'

# API 호출
data = fetch_data(endpoint)
board_nums = []
# 테이블 및 컬럼 생성성
try:
    if connection.is_connected():
        cursor = connection.cursor()
        
        # 1. 게시판 정보 테이블 생성
        create_table_query = """
        CREATE TABLE IF NOT EXISTS board_info (
            id INT AUTO_INCREMENT PRIMARY KEY,
            site_nm VARCHAR(100) NOT NULL,
            board_no VARCHAR(20) NOT NULL,
            board_nm VARCHAR(200) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        cursor.execute(create_table_query)
        connection.commit()
        
        insert_query = """
            INSERT INTO board_info (site_nm, board_no, board_nm)
            VALUES (%s, %s, %s)
            """
        # 2. API 응답 데이터를 DB에 저장
        if len(data) > 0:
            # 데이터를 순회하며 DB에 저장
            for board in data:
                board_nums.append(board['board_no'])
                
                values = (
                    board['site_nm'].strip(),
                    board['board_no'],
                    board['board_nm']
                )
                cursor.execute(insert_query, values)
            
            connection.commit()
            print(f"{len(data)}개의 게시판 정보가 DB에 저장되었습니다.")
        
except Error as e:
    print(f"MySQL 연결/처리 중 에러 발생: {e}")
    
finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")

### CNU 게시판 내용

In [4]:
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
  Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import re
from datetime import datetime

# 1. 게시글 내용 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS board_contents (
    id INT AUTO_INCREMENT PRIMARY KEY,
    board_no INT NOT NULL,
    article_no INT NOT NULL,
    article_title VARCHAR(500) NOT NULL,
    article_text TEXT,
    writer_nm VARCHAR(100),
    click_cnt INT,
    attach_cnt INT,
    update_dt DATETIME,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
"""
cursor.execute(create_table_query)
connection.commit()

# 2. API 응답 데이터를 DB에 저장
insert_query = """
INSERT INTO board_contents 
(board_no, article_no, article_title, article_text, writer_nm, click_cnt, attach_cnt, update_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [15]:
cursor = connection.cursor()
cursor.execute("SELECT board_no FROM board_info")
board_nums = [row[0] for row in cursor.fetchall()]

In [ ]:
# 진행 상태를 저장할 파일 경로
progress_file = 'board_scraping_progress.txt'

# 이전 진행 상태 확인
try:
    with open(progress_file, 'r') as f:
        last_index = int(f.read().strip())
    board_nums = board_nums[last_index:]
    print(f"이전 진행 상태에서 재개합니다. (인덱스: {last_index})")
except FileNotFoundError:
    last_index = 0
    print("처음부터 시작합니다.")

try:
    for i, p_board_no in enumerate(board_nums, start=last_index):
        endpoint = f'{CNU_API_ENDPOINT}/homepageboardContents?AUTH_KEY={AUTH_KEY}&P_board_no={board_nums[i]}'
        # API 호출
        data = fetch_data(endpoint)
        
        for article in data:
            # HTML 태그 제거
            clean_text = BeautifulSoup(article['article_text'], 'html.parser').get_text()
            clean_text = re.sub(r'\s+', ' ', clean_text).strip()
            
            # 날짜 포맷팅
            update_date = datetime.datetime(
                year=article['update_dt']['year'] + 1900,  # year 값이 1900년 기준이므로 보정
                month=article['update_dt']['month'] + 1,   # month는 0-based이므로 1 추가
                day=article['update_dt']['date'],
                hour=article['update_dt']['hours'],
                minute=article['update_dt']['minutes']
            )
            
            values = (
                article['board_no'],
                article['article_no'],
                article['article_title'],
                clean_text,
                article['writer_nm'],
                article['click_cnt'],
                article['attach_cnt'],
                update_date
            )
            cursor.execute(insert_query, values)
        
        connection.commit()
        print(f"게시판 {p_board_no}: {len(data)}개의 게시글이 DB에 저장되었습니다.")
        
        # 진행 상태 저장
        with open(progress_file, 'w') as f:
            f.write(str(i + 1))

except Error as e:
    print(f"MySQL 연결/처리 중 에러 발생: {e}")
    connection.rollback()

### 수강신청

In [4]:
# 테이블 생성
create_table_query = """
CREATE TABLE IF NOT EXISTS sugang_info (
    id VARCHAR(50) PRIMARY KEY,
    pagecnt VARCHAR(10),
    cnt INT,
    open_yr VARCHAR(4),
    cptn_div VARCHAR(50),
    sbjt_nm VARCHAR(100),
    open_sbjt_no VARCHAR(20),
    open_dclss VARCHAR(10),
    dgn_sbjt_yn VARCHAR(1),
    ogdp_cors VARCHAR(20),
    ogdp_dan VARCHAR(20),
    ogdp_orgn_clsf VARCHAR(20),
    ogdp_shyr INT,
    open_shtm VARCHAR(20),
    open_sust_mj VARCHAR(100),
    tlsn_aply_dt VARCHAR(50),
    sugang_count INT DEFAULT 0,
    api_call_idx VARCHAR(20)
)"""

cursor.execute(create_table_query)
connection.commit()

In [ ]:
import os
# 진행상태 파일
progress_file = 'sugang_progress.txt'

# 진행상태 복구
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        p_yr, page, idx = map(int, f.read().split(','))
else:
    p_yr = 2023
    page = 1
    idx = 0

try:
    while p_yr <= 2025:
        # 첫 페이지에서 전체 페이지 수 확인
        endpoint = f'{CNU_API_ENDPOINT}/SugangInfo?AUTH_KEY={AUTH_KEY}&P_OPEN_YR={p_yr}&page={page}'
        data = fetch_data(endpoint)
        if not data:
            continue
            
        total_pages = int(data[0]['PAGECNT'])
        
        while page <= total_pages:
            try:
                if page > 1:
                    endpoint = f'{CNU_API_ENDPOINT}/SugangInfo?AUTH_KEY={AUTH_KEY}&P_OPEN_YR={p_yr}&page={page}'
                    data = fetch_data(endpoint)
            
                for idx, item in enumerate(data, start=idx):
                    # ID 생성
                    id = f"{item['OPEN_YR']}-{item['OPEN_SHTM']}-{item['OPEN_DCLSS']}_{item['OPEN_SBJT_NO']}"
                    api_call_idx = f"{p_yr}{page}-{idx}"
                    
                    # 중복 체크 및 처리
                    check_query = "SELECT id FROM sugang_info WHERE id = %s"
                    cursor.execute(check_query, (id,))
                    
                    if cursor.fetchone():
                        # 중복된 경우 카운트 증가 및 api_call_idx 업데이트
                        update_query = """
                        UPDATE sugang_info 
                        SET sugang_count = sugang_count + 1,
                            api_call_idx = %s 
                        WHERE id = %s
                        """
                        cursor.execute(update_query, (api_call_idx, id))
                    else:
                        # 새로운 데이터 삽입
                        insert_query = """
                        INSERT INTO sugang_info VALUES (
                            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                        )"""
                        values = (
                            id, item['PAGECNT'], item['CNT'], item['OPEN_YR'],
                            item['CPTN_DIV'], item['SBJT_NM'], item['OPEN_SBJT_NO'],
                            item['OPEN_DCLSS'], item['DGN_SBJT_YN'], item['OGDP_CORS'],
                            item['OGDP_DAN'], item['OGDP_ORGN_CLSF'], item['OGDP_SHYR'],
                            item['OPEN_SHTM'], item['OPEN_SUST_MJ'], item['TLSN_APLY_DT'],
                            0, api_call_idx
                        )
                        cursor.execute(insert_query, values)
                
                # 현재 페이지의 모든 처리가 성공하면 커밋
                connection.commit()
                print(f"년도 {p_yr}, 페이지 {page}/{total_pages} 처리 완료")
                
                # 진행상태 저장
                with open(progress_file, 'w') as f:
                    f.write(f"{p_yr},{page},{idx}")
                    
                page += 1
                idx = 0  # 페이지가 바뀌면 idx 초기화
                
            except Exception as e:
                print(f"에러 발생 (년도: {p_yr}, 페이지: {page}): {str(e)}")
                # 현재 페이지의 트랜잭션만 롤백
                raise
                
        # 년도 증가, 페이지 초기화
        p_yr += 1
        page = 1
        idx = 0  # 년도가 바뀌면 idx 초기화
        
except Exception as e:
    print(f"처리 중 에러 발생: {str(e)}")
    # 진행상태는 이미 저장되어 있으므로 다음 실행시 이어서 처리 가능


In [6]:
# 데이터베이스에서 CSV로 내보내기
def export_db_to_csv(connection, table_name: str):
    """
    데이터베이스 테이블의 내용을 CSV 파일로 내보내는 함수
    
    Args:
        connection: MySQL 데이터베이스 연결 객체
        table_name (str): 내보낼 테이블 이름
        output_file (str): 저장할 CSV 파일명
    """
    
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            database='cnu_data',
            user='root',
            password='dydrkfl#7!'
        )
        
        cursor = connection.cursor(dictionary=True)
        
        # 테이블의 모든 데이터 조회
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        
        if not rows:
            print(f"테이블 {table_name}에 데이터가 없습니다.")
            return
            
        # 딕셔너리 형태로 변환
        data = {key: [] for key in rows[0].keys()}
        for row in rows:
            for key, value in row.items():
                data[key].append(value)
                
        # CSV 파일로 저장
        dict_to_csv(data, table_name)
        
        cursor.close()
        print(f"데이터베이스 테이블 {table_name}을(를) CSV 파일로 내보내기 완료")
        
    except Exception as e:
        print(f"데이터베이스 내보내기 중 에러 발생: {e}")

# 수강 정보 테이블을 CSV 파일로 내보내기
export_db_to_csv(connection, "lecture_plan")
export_db_to_csv(connection, "sugang_info")
export_db_to_csv(connection, "board_info")



CSV 파일이 성공적으로 생성되었습니다: lecture_plan
데이터베이스 테이블 lecture_plan을(를) CSV 파일로 내보내기 완료
CSV 파일이 성공적으로 생성되었습니다: sugang_info
데이터베이스 테이블 sugang_info을(를) CSV 파일로 내보내기 완료
CSV 파일이 성공적으로 생성되었습니다: board_info
데이터베이스 테이블 board_info을(를) CSV 파일로 내보내기 완료


In [25]:
import pandas as pd
import os

def create_table_from_csv(connection, csv_file_path: str, table_name: str):
    """
    CSV 파일의 구조를 기반으로 데이터베이스 테이블을 생성하는 함수
    """
    try:
        df = pd.read_csv(csv_file_path)
        cursor = connection.cursor()
        
        # NaN 값을 None으로 변환
        df = df.replace({pd.NA: None, pd.NaT: None, 'nan': None})
        df = df.where(pd.notnull(df), None)
        
        # 컬럼 타입 매핑
        type_mapping = {
            'int64': 'INT',
            'float64': 'FLOAT', 
            'object': 'TEXT',
            'datetime64[ns]': 'DATETIME'
        }
        
        # CREATE TABLE 쿼리 생성
        columns = []
        for col in df.columns:
            col_type = type_mapping.get(str(df[col].dtype), 'TEXT')
            columns.append(f"`{col}` {col_type}")
            
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(columns)}
        )
        """
        
        cursor.execute(create_table_query)
        connection.commit()
        print(f"테이블 {table_name} 생성 완료")
        
    except Exception as e:
        print(f"테이블 생성 중 에러 발생: {e}")
        raise e

def insert_csv_to_db(connection, csv_file_path: str, table_name: str):
    """
    CSV 파일의 데이터를 데이터베이스 테이블에 삽입하는 함수
    """
    try:
        # 진행상황 파일 경로
        progress_file = f"{table_name}_import_progress.txt"
        
        # 이전 진행상황 확인
        start_idx = 0
        if os.path.exists(progress_file):
            with open(progress_file, 'r') as f:
                start_idx = int(f.read().strip() or 0)
        
        df = pd.read_csv(csv_file_path)
        
        # NaN 값과 'nan' 문자열을 None으로 변환
        df = df.replace({pd.NA: None, pd.NaT: None})
        df = df.replace('nan', None) # 문자열 'nan'을 별도로 처리
        df = df.where(pd.notnull(df), None)
        
        cursor = connection.cursor()
        
        # 청크 단위로 데이터 삽입
        chunk_size = 1000
        total_rows = len(df)
        
        for i in range(start_idx, total_rows, chunk_size):
            chunk = df.iloc[i:min(i+chunk_size, total_rows)]
            
            # INSERT 쿼리 생성
            placeholders = ', '.join(['%s'] * len(df.columns))
            columns = '`, `'.join(df.columns)
            insert_query = f"INSERT INTO {table_name} (`{columns}`) VALUES ({placeholders})"
            
            # 데이터 삽입
            values = [tuple(None if pd.isna(x) else x for x in row) for row in chunk.values]
            cursor.executemany(insert_query, values)
            connection.commit()
            
            # 진행상황 저장
            with open(progress_file, 'w') as f:
                f.write(str(i + chunk_size))
                
            print(f"진행률: {min((i + chunk_size) / total_rows * 100, 100):.2f}%")
            
        print(f"CSV 파일 {csv_file_path}의 데이터를 테이블 {table_name}에 삽입 완료")
        
        # 작업 완료 후 진행상황 파일 삭제
        if os.path.exists(progress_file):
            os.remove(progress_file)
            
    except Exception as e:
        print(f"데이터 삽입 중 에러 발생: {e}")
        raise e

def csv_to_db(csv_file_path: str, table_name: str):
    """
    CSV 파일을 데이터베이스 테이블로 변환하는 메인 함수
    """
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            database='cnu_data',
            user='root',
            password='dydrkfl#7!'
        )
        
        # 테이블 생성
        create_table_from_csv(connection, csv_file_path, table_name)
        
        # 데이터 삽입
        insert_csv_to_db(connection, csv_file_path, table_name)
        
        connection.close()
        print("데이터베이스 변환 작업 완료")
        
    except Exception as e:
        print(f"데이터베이스 변환 중 에러 발생: {e}")
        if 'connection' in locals():
            connection.close()

In [19]:
# CSV 파일들을 데이터베이스로 변환
# board_info 테이블 변환
print("\nboard_info 테이블 변환 시작...")
try:
    csv_to_db('board_info.csv', 'board_info')
except Exception as e:
    print(f"board_info 테이블 변환 중 오류 발생: {e}")
print("board_info 테이블 변환 완료")


board_info 테이블 변환 시작...
테이블 board_info 생성 완료
진행률: 100.00%
CSV 파일 board_info.csv의 데이터를 테이블 board_info에 삽입 완료
데이터베이스 변환 작업 완료
board_info 테이블 변환 완료


In [20]:
# lecture_plan 테이블 변환  
print("\nlecture_plan 테이블 변환 시작...")
try:
    csv_to_db('lecture_plan.csv', 'lecture_plan')
except Exception as e:
    print(f"lecture_plan 테이블 변환 중 오류 발생: {e}")
print("lecture_plan 테이블 변환 완료")


lecture_plan 테이블 변환 시작...
테이블 lecture_plan 생성 완료
진행률: 2.54%
진행률: 5.08%
진행률: 7.62%
진행률: 10.16%
진행률: 12.70%
진행률: 15.24%
진행률: 17.78%
진행률: 20.32%
진행률: 22.86%
진행률: 25.40%
진행률: 27.94%
진행률: 30.48%
진행률: 33.03%
진행률: 35.57%
진행률: 38.11%
진행률: 40.65%
진행률: 43.19%
진행률: 45.73%
진행률: 48.27%
진행률: 50.81%
진행률: 53.35%
진행률: 55.89%
진행률: 58.43%
진행률: 60.97%
진행률: 63.51%
진행률: 66.05%
진행률: 68.59%
진행률: 71.13%
진행률: 73.67%
진행률: 76.21%
진행률: 78.75%
진행률: 81.29%
진행률: 83.83%
진행률: 86.37%
진행률: 88.91%
진행률: 91.45%
진행률: 93.99%
진행률: 96.53%
진행률: 99.08%
진행률: 100.00%
CSV 파일 lecture_plan.csv의 데이터를 테이블 lecture_plan에 삽입 완료
데이터베이스 변환 작업 완료
lecture_plan 테이블 변환 완료


In [26]:
# sugang_info 테이블 변환
print("\nsugang_info 테이블 변환 시작...")
try:
    csv_to_db('sugang_info.csv', 'sugang_info')
except Exception as e:
    print(f"sugang_info 테이블 변환 중 오류 발생: {e}")
print("sugang_info 테이블 변환 완료")


sugang_info 테이블 변환 시작...
테이블 sugang_info 생성 완료
진행률: 3.93%
진행률: 7.86%


진행률: 11.79%
진행률: 15.71%
진행률: 19.64%
진행률: 23.57%
진행률: 27.50%
진행률: 31.43%
진행률: 35.36%
진행률: 39.29%
진행률: 43.22%
진행률: 47.14%
진행률: 51.07%
진행률: 55.00%
진행률: 58.93%
진행률: 62.86%
진행률: 66.79%
진행률: 70.72%
진행률: 74.64%
진행률: 78.57%
진행률: 82.50%
진행률: 86.43%
진행률: 90.36%
진행률: 94.29%
진행률: 98.22%
진행률: 100.00%
CSV 파일 sugang_info.csv의 데이터를 테이블 sugang_info에 삽입 완료
데이터베이스 변환 작업 완료
sugang_info 테이블 변환 완료


In [36]:
# sugang_info 테이블에서 미사용중인 tlsn_aply_dt와 api_call_idx 컬럼 삭제
print("\nsugang_info 테이블 컬럼 삭제 시작...")
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root', 
        password='dydrkfl#7!',
        database='cnu_data'
    )
    
    with connection.cursor() as cursor:
        # 컬럼 존재 여부 확인 후 삭제
        cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'sugang_info' AND TABLE_SCHEMA = 'cnu_data'")
        columns = [column[0] for column in cursor.fetchall()]
        
        columns_to_drop = ['tlsn_aply_dt', 'api_call_idx', 'cnt', 'pagecnt']
        existing_columns = [col for col in columns_to_drop if col in columns]
        
        if existing_columns:
            drop_query = f"ALTER TABLE sugang_info DROP COLUMN {', DROP COLUMN '.join(existing_columns)}"
            cursor.execute(drop_query)
            connection.commit()
            print(f"컬럼 삭제 완료: {', '.join(existing_columns)}")
        else:
            print("삭제할 컬럼이 존재하지 않습니다.")
        
    connection.close()

except Exception as e:
    print(f"컬럼 삭제 중 오류 발생: {e}")
    if 'connection' in locals():
        connection.close()
    print(f"컬럼 삭제 중 오류 발생: {e}")
    if 'connection' in locals():
        connection.close()



sugang_info 테이블 컬럼 삭제 시작...
삭제할 컬럼이 존재하지 않습니다.


In [31]:
# kocw_lecture 테이블 변환
print("\nkocw_lecture 테이블 변환 시작...")
try:
    csv_to_db('kocw_lecture_utf8.csv', 'kocw_lecture')
except Exception as e:
    print(f"kocw_lecture 테이블 변환 중 오류 발생: {e}")
print("kocw_lecture 테이블 변환 완료")


kocw_lecture 테이블 변환 시작...
테이블 kocw_lecture 생성 완료
진행률: 31.28%
진행률: 62.56%
진행률: 93.84%
진행률: 100.00%
CSV 파일 kocw_lecture_utf8.csv의 데이터를 테이블 kocw_lecture에 삽입 완료
데이터베이스 변환 작업 완료
kocw_lecture 테이블 변환 완료


In [38]:
# major_interview 테이블 변환 
print("\nmajor_interview 테이블 변환 시작...")
try:
    # major_professor와 major_interview 테이블 생성을 위한 함수 정의
    def create_major_tables():
        try:
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='dydrkfl#7!', 
                database='cnu_data'
            )
            cursor = connection.cursor()

            # 기존 테이블이 있다면 삭제
            cursor.execute("DROP TABLE IF EXISTS major_interview")
            cursor.execute("DROP TABLE IF EXISTS major_professor")

            # major_professor 테이블 생성
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS major_professor (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    number INT,
                    interview_title VARCHAR(200),
                    professor_position VARCHAR(100)
                ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci
            """)

            # major_interview 테이블 생성 
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS major_interview (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    professor_id INT,
                    question TEXT,
                    answer TEXT, 
                    sequence INT,
                    FOREIGN KEY (professor_id) REFERENCES major_professor(id)
                ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci
            """)

            connection.commit()
            print("테이블 생성 완료")

        except Exception as e:
            print(f"테이블 생성 중 오류 발생: {e}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()

    def process_csv_data():
        try:
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='dydrkfl#7!',
                database='cnu_data'
            )
            cursor = connection.cursor()

            # CSV 파일 읽기
            df = pd.read_csv('major_interview_utf8.csv', encoding='utf-8')

            # 각 행 처리
            for index, row in df.iterrows():
                # major_professor 데이터 삽입
                insert_prof_sql = """
                    INSERT INTO major_professor (number, interview_title, professor_position)
                    VALUES (%s, %s, %s)
                """
                prof_values = (
                    int(row['연번']) if pd.notna(row['연번']) else None,
                    str(row['인터뷰제목']) if pd.notna(row['인터뷰제목']) else None,
                    str(row['인터뷰이 직업(직위)']) if pd.notna(row['인터뷰이 직업(직위)']) else None
                )
                cursor.execute(insert_prof_sql, prof_values)
                professor_id = cursor.lastrowid

                # 질문-답변 쌍 처리
                for sequence in range(1, 19):  # 1부터 18까지
                    q_col = f'질문_{sequence}'
                    a_col = f'답변_{sequence}'
                    
                    if q_col in row.index and pd.notna(row[q_col]):
                        insert_qa_sql = """
                            INSERT INTO major_interview (professor_id, question, answer, sequence)
                            VALUES (%s, %s, %s, %s)
                        """
                        qa_values = (
                            professor_id,
                            str(row[q_col]) if pd.notna(row[q_col]) else None,
                            str(row[a_col]) if pd.notna(row[a_col]) else None,
                            sequence
                        )
                        cursor.execute(insert_qa_sql, qa_values)

            connection.commit()
            print("데이터 변환 완료")

        except Exception as e:
            print(f"데이터 처리 중 오류 발생: {e}")
        finally:
            if connection.is_connected():
                cursor.close() 
                connection.close()

    # 테이블 생성 및 데이터 처리 실행
    create_major_tables()
    process_csv_data()

except Exception as e:
    print(f"major_interview 테이블 변환 중 오류 발생: {e}")
print("major_interview 테이블 변환 완료")


major_interview 테이블 변환 시작...
테이블 생성 완료
데이터 변환 완료
major_interview 테이블 변환 완료


In [41]:

try:
    cursor = connection.cursor()

    # departments 테이블 생성
    create_departments_table = """
    CREATE TABLE IF NOT EXISTS departments (
        id INT AUTO_INCREMENT PRIMARY KEY,
        college VARCHAR(100),
        department VARCHAR(100),
        UNIQUE KEY college_dept (college, department)
    )
    """
    cursor.execute(create_departments_table)

    # lecture_plan에서 distinct college, department 조회 후 departments 테이블에 삽입
    insert_departments = """
    INSERT IGNORE INTO departments (college, department)
    SELECT DISTINCT college, department 
    FROM lecture_plan
    WHERE college IS NOT NULL 
    AND department IS NOT NULL
    """
    cursor.execute(insert_departments)
    
    connection.commit()
    print("departments 테이블 생성 및 데이터 삽입 완료")

except Exception as e:
    print(f"departments 테이블 처리 중 오류 발생: {e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


departments 테이블 생성 및 데이터 삽입 완료
